In [9]:
import numpy as np 
import pandas as pd 
import warnings

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

warnings.filterwarnings("ignore",  category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning,)
warnings.filterwarnings("ignore", category=UserWarning)


/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


In [10]:
train_df = pd.read_csv('/kaggle/input/playground-series-s5e7/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s5e7/test.csv')

In [11]:
train_df['Stage_fear'] = train_df['Stage_fear'].mask(train_df['Stage_fear'].isna() & train_df['Drained_after_socializing']
                                            .notna(), train_df['Drained_after_socializing'])
train_df['Drained_after_socializing'] = train_df['Drained_after_socializing'].mask(train_df['Drained_after_socializing']
                                            .isna() & train_df['Stage_fear'].notna(), train_df['Stage_fear'])

In [12]:
test_df['Stage_fear'] = test_df['Stage_fear'].mask(test_df['Stage_fear'].isna() & test_df['Drained_after_socializing']
                                            .notna(), test_df['Drained_after_socializing'])
test_df['Drained_after_socializing'] = test_df['Drained_after_socializing'].mask(test_df['Drained_after_socializing']
                                            .isna() & test_df['Stage_fear'].notna(), test_df['Stage_fear'])

In [13]:
train_df.columns

Index(['id', 'Time_spent_Alone', 'Stage_fear', 'Social_event_attendance',
       'Going_outside', 'Drained_after_socializing', 'Friends_circle_size',
       'Post_frequency', 'Personality'],
      dtype='object')

In [24]:
cols_to_convert = ['Time_spent_Alone', 'Social_event_attendance','Going_outside',  'Friends_circle_size','Post_frequency']
train_df[cols_to_convert] = train_df[cols_to_convert].astype('Int64')
test_df[cols_to_convert] = test_df[cols_to_convert].astype('Int64')




In [25]:
cat_cols = ['Stage_fear','Drained_after_socializing']  # <-- tu wpisz swoje kolumny kategoryczne

train_df[cat_cols]=train_df[cat_cols].fillna('Missing').astype(str)
test_df[cat_cols] = test_df[cat_cols].fillna('Missing').astype(str)



train_df['Sum'] = train_df['Social_event_attendance']+train_df['Going_outside']+train_df['Friends_circle_size']+train_df['Post_frequency']
test_df['Sum'] = test_df['Social_event_attendance']+test_df['Going_outside']+test_df['Friends_circle_size']+test_df['Post_frequency']


In [27]:
from catboost import CatBoostClassifier

cat_features = ['Stage_fear','Drained_after_socializing']

model = CatBoostClassifier(
    cat_features=cat_features,
    iterations=3000,
    learning_rate=0.02,
    depth=5,
    verbose=100,
    random_seed=42
)

# Zakładamy, że target to np. 'label'
model.fit(train_df.drop(['id','Personality'], axis=1), train_df['Personality'])

# Predykcja
preds = model.predict(test_df.drop('id', axis=1))

submission = pd.DataFrame({
    'id': test_df['id'],
    'Personality': preds
})

submission.to_csv("submission.csv", index=False)

print('Sukcesss!!!')

0:	learn: 0.6445539	total: 16.5ms	remaining: 16.5s
100:	learn: 0.1268719	total: 1.46s	remaining: 13s
200:	learn: 0.1231635	total: 2.84s	remaining: 11.3s
300:	learn: 0.1210148	total: 4.23s	remaining: 9.82s
400:	learn: 0.1180096	total: 5.68s	remaining: 8.48s
500:	learn: 0.1152046	total: 7.15s	remaining: 7.12s
600:	learn: 0.1129262	total: 8.63s	remaining: 5.73s
700:	learn: 0.1110000	total: 10.1s	remaining: 4.3s
800:	learn: 0.1089849	total: 11.6s	remaining: 2.89s
900:	learn: 0.1071485	total: 13.1s	remaining: 1.44s
999:	learn: 0.1052823	total: 14.6s	remaining: 0us
Sukcesss!!!
